In [3]:
import pandas as pd
import numpy as np

# 加载 "problem4.csv" 数据
data = pd.read_csv('problem4.csv')
# 显示数据的前几行，确认加载是否正确
print(data.head())

          A         B         C
0  0.004782  0.021225  0.125311
1  0.006932 -0.002660  0.039072
2 -0.005504 -0.005403  0.046670
3 -0.013573 -0.017365  0.078572
4 -0.010876 -0.005402 -0.025213


a. Calculate a covariance matrix from the returns using an exponential weighting factor of
0.94

In [4]:
# 4a. 计算协方差矩阵

# 定义指数加权因子
lambda_ewma = 0.94

# 计算收益矩阵的列数
returns = data.values
num_assets = returns.shape[1]

# 初始化加权协方差矩阵
cov_matrix = np.zeros((num_assets, num_assets))

# 计算指数加权协方差矩阵
for i in range(num_assets):
    for j in range(num_assets):
        cov_matrix[i, j] = np.sum(
            (1 - lambda_ewma) * lambda_ewma**np.arange(len(returns)-1, -1, -1) *
            (returns[:, i] - np.mean(returns[:, i])) *
            (returns[:, j] - np.mean(returns[:, j]))
        )

print("指数加权协方差矩阵:")
print(cov_matrix)


指数加权协方差矩阵:
[[ 1.03874913e-04  1.11695277e-04 -6.42311019e-05]
 [ 1.11695277e-04  3.78311260e-04  1.16468680e-04]
 [-6.42311019e-05  1.16468680e-04  3.23400107e-03]]


b. Calculate the ex-ante risk contribution of each asset as a percent of the total. (

In [5]:
# 4b. 计算风险贡献百分比

# 投资组合权重
weights = np.array([0.30, 0.45, 0.25])

# 组合总风险
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
portfolio_std = np.sqrt(portfolio_variance)

# 计算每个资产的边际风险贡献
marginal_risk_contribution = np.dot(cov_matrix, weights)

# 每个资产的风险贡献
risk_contribution = weights * marginal_risk_contribution

# 每个资产的风险贡献占比
risk_contribution_percent = risk_contribution / portfolio_variance * 100

print("每个资产的风险贡献占比 (%):")
print(risk_contribution_percent)


每个资产的风险贡献占比 (%):
[ 5.85712834 31.29820168 62.84466999]


c. Assuming the returns are daily, the expected returns are the mean (unweighted) of the
historical series, and the risk free rate is 5.25% annually (365 days), what is the daily
Sharpe Ratio of this portfolio?

In [6]:
# 4c. 计算夏普比率

# 计算每日的组合预期收益
daily_expected_returns = np.mean(returns, axis=0)
portfolio_expected_return = np.dot(weights, daily_expected_returns)

# 风险自由利率（每日）
risk_free_rate_daily = 0.0525 / 365

# 夏普比率
sharpe_ratio = (portfolio_expected_return - risk_free_rate_daily) / portfolio_std

print(f"组合的每日夏普比率: {sharpe_ratio}")


组合的每日夏普比率: 0.1996515668882902


d. Calculate the maximum Sharpe Ratio portfolio weights restricting weights to be positive

In [7]:
from scipy.optimize import minimize

# 4d. 计算最大夏普比率的权重

# 定义目标函数：负的夏普比率
def neg_sharpe(weights, returns, risk_free_rate):
    portfolio_return = np.dot(weights, np.mean(returns, axis=0))
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return -sharpe_ratio

# 约束条件：权重之和为1，且每个权重为正
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = [(0, 1) for _ in range(num_assets)]

# 初始猜测权重
initial_weights = np.array([1 / num_assets] * num_assets)

# 优化
result = minimize(
    neg_sharpe,
    initial_weights,
    args=(returns, risk_free_rate_daily),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

max_sharpe_weights = result.x

print("最大夏普比率的权重:")
print(max_sharpe_weights)


最大夏普比率的权重:
[8.12129368e-17 9.64850038e-01 3.51499618e-02]
